中介者模式

解除对象与对象之间的紧耦合关系。

In [2]:
(function(){
    function Player(name){
        this.name = name;
        this.enemy = null;
    }
    
    Player.prototype.win = function(){
        console.log(`${this.name} won`)
    }
    
    Player.prototype.lose = function(){
        console.log(`${this.name} lose`)
    }
    
    Player.prototype.die = function(){
        console.log(`${this.name} die`)
        this.lose()
        this.enemy.win()
    }
    
    var play1 = new Player('no1'),
        play2 = new Player('no2');
    
    play1.enemy = play2;
    play2.enemy = play1;
    play1.die()
})()

no1 die
no1 lose
no2 won


In [5]:
(function(){
    
    var players = [];
    
    function Player(name,teamColor){
        this.partners = [];
        this.enemies = [];
        this.state = 'live';
        this.name = name;
        this.teamColor = teamColor;
    }
    
    Player.prototype.win = function(){
        console.log(`winner: ${this.name}`)
    }
    Player.prototype.lose = function(){
        console.log(`loser: ${this.name}`)
    }
    
    Player.prototype.die = function(){
        var all_dead = true;
        this.state = 'dead';
        
        for(var i =0,partner;partner = this.partners[i++];){
            if(partner.state !== 'dead'){
                all_dead = false;
                break;
            }
        }
        
        if(all_dead === true){
            this.lose()
            for(var i=0,partner;partner = this.partners[i++];){
                partner.lose()
            }
            for(var i =0,enemy;enemy = this.enemies[i++];){
                enemy.win()
            }
        }
    }
    
    var playerFactory = function(name,teamColor){
        var newPlayer = new Player(name,teamColor);
        for(var i =0,player;player = players[i++];){
            if(player.teamColor === newPlayer.teamColor){
                player.partners.push(newPlayer);
                newPlayer.partners.push(player);
            }else{
                player.enemies.push(newPlayer);
                newPlayer.enemies.push(player);
            }            
        }
        players.push(newPlayer);
        
        return newPlayer;
    }
    
    var p1 = playerFactory('pn1','red'),
        p2 = playerFactory('pn2','red'),
        p3 = playerFactory('pn3','red'),
        p4 = playerFactory('pn4','red');
    var n1 = playerFactory('nn1','blue'),
        n2 = playerFactory('nn2','blue'),
        n3 = playerFactory('nn3','blue'),
        n4 = playerFactory('nn4','blue');
    
    p1.die()
    p2.die()
    p3.die()
    p4.die()
})()

loser: pn4
loser: pn1
loser: pn2
loser: pn3
winner: nn1
winner: nn2
winner: nn3
winner: nn4


In [11]:
(function(){
    function Player(name,teamColor){
        this.name = name;
        this.teamColor = teamColor;
        this.state = 'alive';
    }
    
    Player.prototype.win = function(){
        console.log(`${this.name} won`)
    }
    
    Player.prototype.lose = function(){
        console.log(`${this.name} lose`)
    }
    
    Player.prototype.die = function(){
        this.state = 'dead';
        playerDirector.ReceiveMesssage('playerDead',this);
    }
    
    Player.prototype.remove = function(){
        playerDirector.ReceiveMesssage('removePlayer',this);
    }
    
    Player.prototype.changeTeam = function(color){
        playerDirector.ReceiveMesssage('changeTeam',this, color);
    }
    
    var playerFactory = function(name,teamColor){
        var newPlayer = new Player(name,teamColor);
        playerDirector.ReceiveMesssage('addPlayer',newPlayer);
        return newPlayer;
    }
    
    var playerDirector = (function(){
        var players = {},
            operations = {};
        
        operations.addPlayer = function(player){
            var teamColor = player.teamColor;
            players[teamColor] = players[teamColor]||[];
            players[teamColor].push(player);
        }
        
        operations.removePlayer = function(player){
            var teamColor = player.teamColor,
                teamPlayers = players[teamColor]||[];
            for(var i = teamPlayers.length -1;i>=0;i--){
                if(teamPlayers[i]===player){
                    teamPlayers.splice(i,1)
                }
            }
        }
        
        operations.changeTeam = function(player,newTeamColor){
            operations.removePlayer(player);
            player.teamColor = newTeamColor;
            operations.addPlayer(player);
        }
        
        operations.playerDead = function(player){
            var teamColor = player.teamColor,
                teamPlayers = players[teamColor];
            
            var all_dead = true;
            for(var i =0,player;player = teamPlayers[i++];){
                if(player.state !== 'dead'){
                    all_dead = false;
                    break;
                }
            }
            
            if(all_dead === true){
                for(var i =0,player;player = teamPlayers[i++];){
                    player.lose()
                }
                
                for(var color in players){
                    if(color !== teamColor){
                        var teamPlayers = players[color];
                        for(var i =0,player;player = teamPlayers[i++];){
                            player.win()
                        }
                    }
                }
            }
        }
        
        var ReceiveMessage = function(){
            var message = Array.prototype.shift.call(arguments);
            operations[message].apply(this,arguments);
        }
        
        return {
            ReceiveMesssage:ReceiveMessage
        }
    })()
    
    var p1 = playerFactory('pn1','red'),
        p2 = playerFactory('pn2','red'),
        p3 = playerFactory('pn3','red'),
        p4 = playerFactory('pn4','red');
    var n1 = playerFactory('nn1','blue'),
        n2 = playerFactory('nn2','blue'),
        n3 = playerFactory('nn3','blue'),
        n4 = playerFactory('nn4','blue');
    
    p1.remove()
    p2.changeTeam('blue')
    p3.die()
    p4.die()
})()

pn3 lose
pn4 lose
nn1 won
nn2 won
nn3 won
nn4 won
pn2 won


中介者模式是迎合迪米特法则的一种实现。
迪米特法则也叫最少知识法则，指一个对象应该尽可能少的了解另外的对象。